# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [16]:
!pip install ultralytics

In [17]:
import cv2
import numpy as np
from ultralytics import YOLO

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [18]:
model = YOLO('/content/YOLO_Model.pt', task='detect')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [19]:
video_path = '/content/YOLO_Video (2).mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [20]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [21]:
print(height ,width ,fps)

1080 1920 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [22]:
output_path = '/content/output_video_with_predictions.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [23]:
ret,frame = cap.read()
results = model(frame)


0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2964.6ms
Speed: 3.6ms preprocess, 2964.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)


## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [15]:
width = int(cap.get(3))
height = int(cap.get(4))
top_roi = (0, 0, width, 50)
bottom_roi = (0, height - 50, width, height)
left_roi = (0, 0, 50, height)
right_roi = (width - 50, 0, width, height)

# Initialize counters
top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

while cap.isOpened():
    success, frame = cap.read()  # Read a frame
    if success:
        results = model(frame)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()

            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes[i]
                confidence = confidences[i]
                class_id = classes[i]

                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # Draw bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                # Draw confidence score and class ID
                text = f"Conf: {confidence:.2f}, Class: {class_id}"
                cv2.putText(frame, text, (int(x1), int(y1) - 10), cv2.FONT_ITALIC, 0.5, (0, 255, 0), 2)

                # Draw center of the bounding box
                cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

                # Check if the vehicle exited from any street
                if center_y < top_roi[3]:
                    top_count += 1
                if center_y > bottom_roi[1]:
                    bottom_count += 1
                if center_x < left_roi[2]:
                    left_count += 1
                if center_x > right_roi[0]:
                    right_count += 1

        # Draw ROIs
        cv2.rectangle(frame, (top_roi[0], top_roi[1]), (top_roi[2], top_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (bottom_roi[0], bottom_roi[1]), (bottom_roi[2], bottom_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (left_roi[0], left_roi[1]), (left_roi[2], left_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (right_roi[0], right_roi[1]), (right_roi[2], right_roi[3]), (0, 255, 0), 2)

        # Display counts
        cv2.putText(frame, f"Top: {top_count}", (10, 30), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Bottom: {bottom_count}", (10, 60), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Left: {left_count}", (10, 90), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Right: {right_count}", (10, 120), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

        # Display your name on top of the video
        cv2.putText(frame, "Hanin", (10, height - 10), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

        # Write frame to output video
        out.write(frame)
    else:
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved to {output_path}")

Video saved to /content/output_video_with_predictions.mp4


https://drive.google.com/file/d/1H0fx_VNQisXiaC6wJoiUK-dpY60V8ed5/view?usp=drive_link

In [ ]:
top_roi = (0, 0, width, 50)
bottom_roi = (0, height - 50, width, height)
left_roi = (0, 0, 50, height)
right_roi = (width - 50, 0, width, height)
# count
top_count = 0
bottom_count = 0
left_count = 0
right_count = 0


while cap.isOpened():

    success, frame = cap.read()     # Read a frame

    if success:

        results = model(frame)


        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()

            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes[i]
                confidence = confidences[i]
                class_id = classes[i]


                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)


                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)


                text = f"Conf: {confidence:.2f}, Class: {class_id}"
                cv2.putText(frame, text, (int(x1), int(y1) - 10), cv2.FONT_ITALIC, 0.5, (0, 255, 0), 2)


                cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

  # Check if the vehicle exited from any street

                if center_y < top_roi[3]:
                    top_count += 1
                if center_y > bottom_roi[1]:
                    bottom_count += 1
                if center_x < left_roi[2]:
                    left_count += 1
                if center_x > right_roi[0]:
                    right_count += 1


        cv2.rectangle(frame, (top_roi[0], top_roi[1]), (top_roi[2], top_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (bottom_roi[0], bottom_roi[1]), (bottom_roi[2], bottom_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (left_roi[0], left_roi[1]), (left_roi[2], left_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (right_roi[0], right_roi[1]), (right_roi[2], right_roi[3]), (0, 255, 0), 2)


        cv2.putText(frame, f"Top: {top_count}", (10, 30), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Bottom: {bottom_count}", (10, 60), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Left: {left_count}", (10, 90), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Right: {right_count}", (10, 120), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)


        cv2.putText(frame, "Hanin", (10, height - 20), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)


        out.write(frame)

    else:
        break


cap.release()
out.release()
cv2.destroyAllWindows()
cap.release()
out.release()
print(f"Video saved to {output_path}")



0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 2641.7ms
Speed: 4.4ms preprocess, 2641.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 3216.7ms
Speed: 4.4ms preprocess, 3216.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3328.9ms
Speed: 9.2ms preprocess, 3328.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2657.6ms
Speed: 5.7ms preprocess, 2657.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2651.3ms
Speed: 3.9ms preprocess, 2651.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2668.3ms
Speed: 7.0ms preprocess, 2668.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3866.7ms
Speed: 4.4ms p

## Save and Submit Your Work

Submit both the notebook and the output video